In [1]:
import pandas as pd
import sys

sys.path.append("..")
from data_generation_scripts.utils import check_rate_limit, get_core_users_repos
from data_generation_scripts.generate_user_repos_interactions import get_user_repo_activities
from data_generation_scripts.generate_user_users_interactions import get_user_users_activities

In [2]:
rates_df = check_rate_limit()

### Get Initial Users

In [3]:
core_users, core_repos = get_core_users_repos()

### Get User Followers

In [9]:
user_users_output_path = "../data/join_files/user_followers_join_dataset.csv"
users_output_path = "../data/entity_files/users_dataset.csv"
get_url_field = "followers_url"
load_existing_files = True
overwrite_existing_temp_files = False

users_followers_df, user_df = get_user_users_activities(core_users,user_users_output_path, users_output_path, get_url_field, load_existing_files, overwrite_existing_temp_files)

### Get User Following

In [10]:
user_users_output_path = "../data/join_files/user_following_join_dataset.csv"
users_output_path = "../data/entity_files/users_dataset.csv"
get_url_field = "following_url"
load_existing_files = True
overwrite_existing_temp_files = False

users_following_df, user_df = get_user_users_activities(core_users, user_users_output_path, users_output_path, get_url_field, load_existing_files, overwrite_existing_temp_files)

### Get User Stars

In [12]:
user_users_output_path = "../data/large_files/join_files/user_starred_join_dataset.csv"
users_output_path = "../data/entity_files/users_dataset.csv"
get_url_field = "starred_url"
load_existing_files = True
overwrite_existing_temp_files = False

users_starred_df, user_df = get_user_repo_activities(core_users,user_users_output_path, users_output_path, get_url_field, load_existing_files, overwrite_existing_temp_files)

### Get User Repos